In [ ]:
# plot as in Gladyshev, M. (2002). Biophysics of the surface microlayer of aquatic ecosystems. IWA Publishing.
# Pia Goecke pia.goecke@uni-oldenburg.de
# 06.12.2024

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings("ignore")
import os
import seaborn as sns
import plotly.graph_objs as go

%matplotlib qt


In [ ]:
# read HE626
# quality Thies
# 

In [3]:
# read data from cruise HE614
#read data and rename cols
os.chdir('C:/Users/piago/Documents/Uni/5_Semester/Hiwi')
csv_file = "C:/Users/piago/Documents/Uni/5_Semester/Hiwi/Data/HE614_All_Grids_Single_Sheet.xlsx"

# Read the Excel file
df = pd.read_excel(csv_file, skiprows=10)
df = df.set_index('Date_Time')
df.index = pd.to_datetime(df.index)
df.rename(columns={'CTD_Temperature_50cm_[°C]' : 'CTD_Temperature_50cm',
                   'CTD_Temperature_60cm_[°C]': 'CTD_Temperature_60cm'
                   }, inplace= True)

# df['Date'] = df.index.date
df_614 = df



In [4]:
# read data from HE598
csv_file = "C:/Users/piago/Documents/Uni/5_Semester/Hiwi/Data/HE598_All_Grids_Single_Sheet_Revised_v1.xlsx"

# Read the Excel file
df = pd.read_excel(csv_file)
df = df.set_index('Date_Time')
df.index = pd.to_datetime(df.index)
df_598 = df

In [5]:
# read data from HE609
csv_file = "C:/Users/piago/Documents/Uni/5_Semester/Hiwi/Data/HE609_All_Grids_Single_Sheet_Revised_v1.xlsx"

# Read the Excel file
df = pd.read_excel(csv_file)
df = df.set_index('Date_Time')
df.index = pd.to_datetime(df.index)
df_609 = df

In [6]:
# read data from HE626
csv_file = "C:/Users/piago/Documents/Uni/5_Semester/Hiwi/Data/HE626_All_Grids_Single_Sheet.xlsx"

# Read the Excel file
df = pd.read_excel(csv_file)
df = df.set_index('Date_Time')
df.index = pd.to_datetime(df.index)
df_626 = df

In [7]:
# some new columns
df_598['cruise'] = 'HE598'
df_609['cruise'] = 'HE609'
df_614['cruise'] = 'HE614'
df_626['cruise'] = 'HE626'

df_598['Date_Time'] = df_598.index
df_609['Date_Time'] = df_609.index
df_614['Date_Time'] = df_614.index
df_626['Date_Time'] = df_626.index

In [8]:
allcruises = pd.concat([df_598, df_609, df_614, df_626], axis = 0, ignore_index=True)
allcruises['Date'] = pd.to_datetime(allcruises['Date_Time'].dt.date)
allcruises['Month'] = allcruises['Date_Time'].dt.month

In [83]:
# interpolate Thies solar and thies_3_temp, grouped by day

allcruises['Thies_Solar_Radiation_SMP6_AVG_interp'] = allcruises.groupby('Date')['Thies_Solar_Radiation_SMP6_AVG'].apply(lambda group: group.interpolate()).reset_index(level=0, drop=True)
allcruises['Thies_Temperature_3_AVG_interp'] = allcruises.groupby('Date')['Thies_Temperature_3_AVG'].apply(lambda group: group.interpolate()).reset_index(level=0, drop=True)
allcruises['Thies_Wind_Speed_CUS_AVG_interp'] = allcruises.groupby('Date')['Thies_Wind_Speed_CUS_AVG'].apply(lambda group: group.interpolate()).reset_index(level=0, drop=True)


In [18]:
grouped = allcruises.groupby('Date') 
for date, data in grouped:
    plt.figure()
    plt.scatter(data['Date_Time'], data['Thies_Temperature_3_AVG'])
    plt.title(data['Date'].iloc[0])
    plt.show()

In [82]:
grouped = allcruises.groupby('Date') 
missing_campbell = grouped['Thies_Wind_Speed_CUS_AVG'].apply(lambda x : x.isna().all()).sum()
missing_Thies_3 = grouped['Thies_Temperature_3_AVG'].apply(lambda x : x.isna().all()).sum()

print(f'number of days with misisng Campbell wind: {missing_campbell} \nnumber of days with missing Thies_3: {missing_Thies_3}')

#14.10., 12.10: npo campbell
# 5.5. no thies


number of days with misisng Campbell wind: 1 
number of days with missing Thies_3: 1


In [72]:
allcruises['sml-bulk'] = allcruises['CTD_Temperature_SML'] - allcruises['CTD_Temperature_100cm']
allcruises['air-bulk'] = allcruises['Campbell_Air_Temperature'] - allcruises['CTD_Temperature_100cm'] # choose airtemp

#Thies_Temperature_3_AVG
#Campbell_Air_Temperature
#Thies_Temperature_3_AVG_interp

In [38]:
# plot SML-bulk / air-bulk

df = allcruises#[allcruises['cruise'] == 'HE626']  # plot everything or only 1 cruise

plt.figure()
sns.scatterplot(data=df, x = 'air-bulk', y = 'sml-bulk', hue = 'Month',     # hue as 'Thies_Solar_Radiation_SMP6_AVG_interp' or 'Wind_Speed_U10', or 'Month', 'Date', ...
                # style = 'Month', 
                palette='viridis', s = 30,
            zorder = 2)

plt.axvline(0, color = 'black', zorder = 1)
plt.axhline(0, color = 'black', zorder = 1)


plt.ylabel('SML-bulk' ,fontsize = 15)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.xlabel('Air-bulk', fontsize = 15)
plt.legend(title = 'Month', fontsize = 15, title_fontsize = 15)
plt.show()

In [75]:
df = allcruises[allcruises['Date'] == '2023-03-17']
plt.figure()
plt.scatter(df.index, df['Campbell_Air_Temperature'])
plt.scatter(df.index, df['Thies_Temperature_3_AVG'])
plt.show()

In [87]:
# find interesting dates
# df = allcruises[allcruises['Date'] == '2022-10-10']  # plot everything or only 1 cruise
df = df_normal_dates
plt.figure()
sns.scatterplot(data=df, x = 'air-bulk', y = 'sml-bulk', hue = 'Thies_Solar_Radiation_SMP6_AVG_interp',     # hue as 'Thies_Solar_Radiation_SMP6_AVG_interp' or 'Wind_Speed_U10', or 'Month', 'Date', ...
                style = 'Month',
                  palette='inferno', s = 30,
            zorder = 2)

plt.axvline(0, color = 'black', zorder = 1)
plt.axhline(0, color = 'black', zorder = 1)


plt.ylabel('SML-bulk' ,fontsize = 15)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.xlabel('Air-bulk', fontsize = 15)
plt.legend(fontsize = 15,
            # title = 'Date',
              title_fontsize = 15)
plt.show()

# 08.03.
# 02.05.
# 10.05.
# 17.05.
# 05.08.
# 10.10.

# check 17.03. air

In [85]:
dates_of_interest = ['2023-03-08', '2022-05-02', '2022-05-10', '2022-05-17',
                      '2023-08-05', 
                      '2022-10-10']
df_dates_of_interest = allcruises[allcruises['Date'].isin(pd.to_datetime(dates_of_interest))]
df_normal_dates = allcruises[~allcruises['Date'].isin(pd.to_datetime(dates_of_interest))]


'''
2023-03-08: thies - long tail with high SML-Air, no CTD:100cm data after 12:30. Thies weird in wind, temp, rad. 
    --> use campbell
2022-05-02: radiation - 1 outliers in beginning, Campbell-airtemp below watertemp, airtemp at 50cm above, check mission plan (west)
    --> cut start, check all thies temps
2022-05-10: high ctd100 in beginning, high thies2 temp in beginning (might be right)
    --> 85cm for bulk?
2022-05-17: +- same grid twice. sml temp negative corr air temp. air 50 cm warmer than 3m,, aml warmer than 100
    --> check all thies temps, check precip
2022-08-05: 
2022-10-10: ctd100 befor 13 -1C off, no change in loc, precip event -> decrease all temps
    --> cut start

2023-03.17: high tif water / air, air much warmer
'''

'\n2023-03-08: thies - long tail with high SML-Air, no CTD:100cm data after 12:30. Thies weird in wind, temp, rad. \n    --> use campbell\n2022-05-02: radiation - 1 outliers in beginning, Campbell-airtemp below watertemp, airtemp at 50cm above, check mission plan (west)\n    --> cut start, check all thies temps\n2022-05-10: high ctd100 in beginning, high thies2 temp in beginning (might be right)\n    --> 85cm for bulk?\n2022-05-17: +- same grid twice. sml temp negative corr air temp. air 50 cm warmer than 3m,, aml warmer than 100\n    --> check all thies temps, check precip\n2022-08-05: \n2022-10-10: ctd100 befor 13 -1C off, no change in loc, precip event -> decrease all temps\n    --> cut start\n\n2023-03.17: high tif water / air, air much warmer\n'

In [122]:
# temperatures
# temperatures = df_dates_of_interest.filter(regex = '^CTD_Temp') # or: 'CTD_Temperature_SML', 'CTD_Temperature_100cm', 'Campbell_Air_Temperature', 'Thies_Temperature_3_AVG'
temperatures = ['CTD_Temperature_SML', 'CTD_Temperature_100cm', 'Campbell_Air_Temperature', 'Thies_Temperature_3_AVG']
grouped = allcruises[allcruises['cruise'] == 'HE626'].groupby('Date') 
 
for date, data in grouped:
    plt.figure(figsize=(8, 6))
    for temp in temperatures: 
        plt.scatter(data['Date_Time'], data[temp], label=f'{temp}', alpha=0.6)
        plt.title(f"Longitude vs Latitude for {date}")
        plt.xlabel('time')
        plt.ylabel('Temperature [°C]')
        plt.grid(True)
        plt.legend()
        plt.show()

In [71]:
# solar radiation
# radiation= df_dates_of_interest.filter(regex = 'Solar').columns.tolist() # or only net radiation: 
radiation = [ 'Solar_Radiation_Net', 'Thies_Solar_Radiation_SMP6_AVG_interp', 'Thies_Solar_Radiation_SMP6_AVG']
grouped = df_dates_of_interest.groupby('Date')
 
for date, data in grouped:
    plt.figure(figsize=(8, 6))
    for rad in radiation: 
        plt.scatter(data['Date_Time'], data[rad], label=f'{rad}', alpha=0.6)
        plt.title(f"Longitude vs Latitude for {date}")
        plt.xlabel('time')
        plt.ylabel('radiation')
        plt.grid(True)
        plt.legend()
        plt.show()


In [77]:
# show all wind things
wind = df_dates_of_interest.filter(regex = 'Wind_Direction').columns.tolist() # regex = 'Wind_Speed', or 'Wind_Direction', or only:
# wind = ['Wind_Speed_U10', 'Thies_Wind_Speed_CUS_AVG']
grouped = df_dates_of_interest.groupby('Date')
 
for date, data in grouped:
    plt.figure(figsize=(8, 6))
    for p in wind: 
        plt.scatter(data.index, data[p], label=f'{p}', alpha=0.6)
        plt.title(f"Longitude vs Latitude for {date}")
        plt.xlabel('time')
        plt.ylabel('windspeed or direction')
        plt.grid(True)
        plt.legend()
        plt.show()

In [131]:

date = '2022-05-17'
df = allcruises[allcruises['Date'] == date]

plt.figure(figsize =(8,6))
plt.scatter(df['Date_Time'], df['sml-bulk'], label = 'SML-Bulk')
plt.scatter(df['Date_Time'], df['air-bulk'], label = 'Air-Bulk')
plt.scatter(df['Date_Time'], df['CTD_Temperature_100cm'], label = 'CTD_100')
plt.scatter(df['Date_Time'], df['CTD_Temperature_SML'], label = 'CTD_SML')
plt.scatter(df['Date_Time'], df['Thies_Temperature_3_AVG'], label = 'Air')
plt.scatter(df['Date_Time'], df['Precipitation_Intensity'], label = 'Air')

plt.ylabel('T-difference' ,fontsize = 15)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.xlabel('Time', fontsize = 15)
plt.legend()
plt.show()


In [76]:
# plot everything for one day
date = '2023-03-17'
df = allcruises[allcruises['Date'] == date]

# chose what to show:

# temperatures = df.filter(regex = '^CTD_Temp') # or:
temperatures = ['CTD_Temperature_SML', 'CTD_Temperature_100cm', 'Campbell_Air_Temperature', 'Thies_Temperature_3_AVG']

# radiation= df.filter(regex = 'Solar').columns.tolist() # or only net radiation: 
radiation = [ 'Solar_Radiation_Net', 'Thies_Solar_Radiation_SMP6_AVG_interp', 'Thies_Solar_Radiation_SMP6_AVG']

# wind = df.filter(regex = 'Wind_Speed').columns.tolist() # regex = 'Wind_Speed', or 'Wind_Direction', or:
wind = ['Wind_Speed_U10', 'Campbell_Wind_Speed','Thies_Wind_Speed_CUS_AVG'] 

precipitation = df.filter(regex = 'Precipitation')




# where was it?
plt.figure(figsize = (8,6))
sns.scatterplot(data=df, x = 'air-bulk', y = 'sml-bulk', hue = 'Date',     # hue as 'Thies_Solar_Radiation_SMP6_AVG_interp' or 'Wind_Speed_U10', or 'Month', 'Date', ...
                style = 'Month', palette='inferno', s = 20,
            zorder = 2)

plt.axvline(0, color = 'black', zorder = 1)
plt.axhline(0, color = 'black', zorder = 1)


plt.ylabel('SML-bulk' ,fontsize = 15)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.xlabel('Air-bulk', fontsize = 15)
plt.show()

# as time series
plt.figure(figsize =(8,6))
plt.scatter(df['Date_Time'], df['sml-bulk'])
plt.scatter(df['Date_Time'], df['air-bulk'])
plt.ylabel('T-difference' ,fontsize = 15)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.xlabel('Time', fontsize = 15)
plt.legend()
plt.show()

# Locations 
plt.figure(figsize=(8, 6))
sc = plt.scatter(df['Longitude'], df['Latitude'], c = df['Date_Time'], label=f'Data for {date}', alpha=0.6)
plt.text(df['Longitude'].min(), df['Latitude'].min(), f'Start: {df.Date_Time.dt.hour.iloc[0]}:{df.Date_Time.dt.minute.iloc[0]}', color = 'red',ha='left', va='center', fontsize=20)
plt.text(df['Longitude'].max(), df['Latitude'].min(), f'End: {df.Date_Time.dt.hour.iloc[-1]}:{df.Date_Time.dt.minute.iloc[-1]}',color = 'blue', ha='right', va='center', fontsize=20)
plt.title(f"Longitude vs Latitude for {date}")
plt.scatter(df['Longitude'].iloc[0], df['Latitude'].iloc[0], s = 100, c = 'red')
plt.scatter(df['Longitude'].iloc[-1], df['Latitude'].iloc[-1], s = 100, c = 'blue')
# cbar = plt.colorbar(sc)
# cbar.set_label('CTD Temperature 30cm [°C]')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.legend()
plt.show()

# Temperature
plt.figure(figsize=(8, 6))
for temp in temperatures: 
    plt.scatter(df['Date_Time'], df[temp], label=f'{temp}', alpha=0.6)
    plt.title(f"Temperatures for {date}")
    plt.xlabel('time')
    plt.ylabel('Temperature [°C]')
    plt.grid(True)
    plt.legend()
    plt.show()

 
# Radiation
plt.figure(figsize=(8, 6))
for rad in radiation: 
    plt.scatter(df['Date_Time'], df[rad], label=f'{rad}', alpha=0.6)
    plt.title(f"Radiation for {date}")
    plt.xlabel('time')
    plt.ylabel('radiation')
    plt.grid(True)
    plt.legend()
    plt.show()

# Wind
plt.figure(figsize=(8, 6))
for p in wind: 
    plt.scatter(df['Date_Time'], df[p], label=f'{p}', alpha=0.6)
    plt.title(f"Wind for {date}")
    plt.xlabel('time')
    plt.ylabel('windspeed or direction')
    plt.grid(True)
    plt.legend()
    plt.show()

# Precipitation
plt.figure(figsize =(8,6))
for precip in precipitation:
    plt.scatter(df['Date_Time'], df[precip], label = f'{precip}')
    plt.title(f"Precipitation for {date}")
    plt.xlabel('time')
    plt.ylabel('Precipitation')
    plt.grid(True)
    plt.legend()
    plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [132]:
# thies-temperatures
thies_sensors = ['CUS', '5', '2', '4', '3']  # CUS and 5 at 200cm, 2 at 150, 4 at 100, 3 at 50
grouped = df_dates_of_interest.groupby('Date')
 
for date, data in grouped:
    plt.figure(figsize=(8, 6))
    plt.scatter(data['Date_Time'], data['Campbell_Air_Temperature'], label = 'Campbell')
    for sensor in thies_sensors: 
        plt.scatter(data['Date_Time'], data[f'Thies_Temperature_{sensor}_AVG'], label=f'{sensor}', alpha=0.6)
        plt.title(f"Thies_temperatures {date}")
        plt.xlabel('time')
        plt.ylabel('Temperature [°C]')
        plt.grid(True)
    plt.legend()
    plt.show()

In [41]:
# Precipitation
precips = ['Precipitation_Intensity', 'Precipitation_Amount']
grouped = df_dates_of_interest.groupby('Date')
 
for date, data in grouped:
    plt.figure(figsize=(8, 6))
    for precip in precips: 
        plt.scatter(data['Date_Time'], data[precip], label=f'{precip}', alpha=0.6)
        plt.title(f"Precipitation {date}")
        plt.xlabel('time')
        plt.ylabel('Precipitation')
        plt.grid(True)
        plt.legend()
        plt.show()

In [ ]:
for date in np.unique(allcruises['Date']):
    df = allcruises[allcruises['Date'] == date]
    try:
        plt.figure()
        plt.hist(df['Quality_Thies_Temperature_5_AVG'])
        plt.show()
    except ValueError:
        print(f'no Quality flags for {date}')

# all quality for Thies Temperature probably good

no Quality flags for 2022-05-05T00:00:00.000000000


In [17]:
df_598_17 = allcruises[allcruises['Date'] == '2022-05-17']
df_598_17_1 = df_598_17[df_598_17['Date_Time'] < '2022-05-17 11:00']
df_598_17_2 = df_598_17[df_598_17['Date_Time'] > '2022-05-17 11:00']

In [18]:
plt.figure()
plt.scatter(df_598_17_1['Longitude'], df_598_17_1['Latitude'], c = df_598_17_1['Date_Time'])
cbar = plt.colorbar(cmap='viridis', pad=0.03)
plt.show()

In [132]:
plt.figure()
plt.scatter(df_598_17_1['Date_Time'], df_598_17_1['Campbell_Air_Temperature'])
plt.scatter(df_598_17_1['Date_Time'], df_598_17_1['Thies_Temperature_3_AVG'])
plt.scatter(df_598_17_1['Date_Time'], df_598_17_1['CTD_Temperature_SML'])

plt.scatter(df_598_17_1['Date_Time'], df_598_17_1['CTD_Temperature_100cm'])

plt.scatter(df_598_17_1['Date_Time'], df_598_17_1['Wind_Speed_U10'])


In [10]:
df = df_598_17_2
plt.figure(figsize=(8, 6))
plt.scatter(df['Longitude'], df['Latitude'], label=f'Data for 17.05 (II)', alpha=0.6)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.legend()
# plt.savefig(f"C:/Users/piago/Documents/Uni/5_Semester/Hiwi/Plots/grid_{date}.png")
plt.show()

NameError: name 'df_598_17_2' is not defined

In [16]:
CTD_Temps = df_598.filter(regex = '^CTD_Temp').columns.tolist()
df = df_598[df_598['Date'] == '2022-05-10']

fig = go.Figure()

for col in CTD_Temps:
    fig.add_trace(go.Scatter(
        x = df['Date_Time'], y = df[col],
        mode = 'lines+markers',
        marker = dict(size=5),
        name = col[-5:-1]
    ))

fig.add_trace(go.Scatter(
x = df['Date_Time'], y = df['Campbell_Air_Temperature'],
mode = 'lines',
marker = dict(size=2),
name = 'Air'
))

fig.add_trace(go.Scatter(
x = df['Date_Time'], y = df['Thies_Temperature_3_AVG'],
mode = 'markers',
marker = dict(size=7),
name = 'Air_thies'
))

fig.add_trace(go.Scatter(
x = df['Date_Time'], y = df['Precipitation_Intensity'],
mode = 'markers',
marker = dict(size=7),
name = 'Precip'
))

fig.update_layout(
    title = 'CTD and Air Temperature',
    xaxis_title = 'Time',
    yaxis_title = 'Temperature [°C]',
    legend = dict(
        traceorder = 'normal'
    )
)

fig.show()

In [19]:
 # plot all lat lons
df = allcruises
grouped = df.groupby('Date')

for date, data in grouped:
    plt.figure(figsize=(8, 6))
    sc = plt.scatter(data['Longitude'], data['Latitude'],  c = data['CTD_Temperature_30cm'], label=f'Data for {date}', alpha=0.6)
    plt.text(data['Longitude'].min(), data['Latitude'].min(), f'Start: {data.Date_Time.dt.hour.iloc[0]}:{data.Date_Time.dt.minute.iloc[0]}', color = 'red',ha='left', va='center', fontsize=20)
    plt.text(data['Longitude'].max(), data['Latitude'].min(), f'End: {data.Date_Time.dt.hour.iloc[-1]}:{data.Date_Time.dt.minute.iloc[-1]}',color = 'blue', ha='right', va='center', fontsize=20)
    plt.title(f"Longitude vs Latitude for {date}")
    plt.scatter(data['Longitude'].iloc[0], data['Latitude'].iloc[0], s = 100, c = 'red')
    plt.scatter(data['Longitude'].iloc[-1], data['Latitude'].iloc[-1], s = 100, c = 'blue')
    cbar = plt.colorbar(sc)
    cbar.set_label('CTD Temperature 30cm [°C]')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.grid(True)
    plt.legend()
    # plt.savefig(f"C:/Users/piago/Documents/Uni/5_Semester/Hiwi/Plots/grid_{date}.png")
    plt.show()

In [25]:
allcruises['Date'][0]

Timestamp('2022-05-02 00:00:00')

In [27]:
allcruises['CTD_Temperature_30cm']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
          ...  
54205    6.1731
54206    6.1769
54207    6.1762
54208    6.1811
54209    6.1857
Name: CTD_Temperature_30cm, Length: 54210, dtype: float64

In [36]:
allcruises.filter(regex = 'RBR_Tem').columns.to_list()

['RBR_Temperature_30cm',
 'RBR_Temperature_40cm',
 'RBR_Temperature_50cm',
 'RBR_Temperature_60cm',
 'RBR_Temperature_85cm',
 'RBR_Temperature_100cm',
 'Quality_RBR_Temperature_30cm',
 'Quality_RBR_Temperature_40cm',
 'Quality_RBR_Temperature_50cm',
 'Quality_RBR_Temperature_60cm',
 'Quality_RBR_Temperature_85cm',
 'Quality_RBR_Temperature_100cm']

In [41]:
depths = ['SML', '30cm', '40cm', '50cm', '60cm', '85cm', '100cm']

for d in depths:
    allcruises[f'CTD_detrend_{d}'] = np.nan

In [42]:

for date in ['2023-03-02','2023-03-08','2023-03-09','2023-03-17' ]:
    df = allcruises[allcruises['Date'] == date]

    # time in total seconds
    df['time_numeric'] = (df['Date_Time'] - df['Date_Time'].min()).dt.total_seconds()  

    # same for all depths
    if date == '2023-03-08':
        depths = ['SML', '30cm', '40cm', '50cm', '60cm', '85cm']
    else:
        depths = ['SML', '30cm', '40cm', '50cm', '60cm', '85cm', '100cm']

    for d in depths:
        if df[f'CTD_Temperature_{d}'].dropna().empty:
            # If the column is empty (all NaNs), skip this depth
            print(f"{date}: Column CTD_Temperature_{d} contains only NaN values. Creating nan-column for depth {d}.")
            df[f'detrend_2_Temp_{d}'] = np.nan

        else:
            coefficients = np.polyfit(df['time_numeric'], df[f'CTD_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'detrend_2_Temp_{d}'] = df[f'CTD_Temperature_{d}'] - df[f'fitted_2_Temp_{d}']
            df[f'detrend_2_Temp_{d}'] = df[f'detrend_2_Temp_{d}'] + df[f'fitted_2_Temp_{d}'].median()
            allcruises.loc[allcruises['Date'] == date, f'CTD_detrend_{d}'] = df[f'detrend_2_Temp_{d}']
            # df.rename(columns={f'CTD_Temperature_{d}' : f'CTD_Temperature_{d}_old'}, inplace=True)
            # df.rename(columns={f'detrend_2_Temp_{d}' : f'CTD_Temperature_{d}'}, inplace=True)


#     # ... and same for airtemp
#     coefficients = np.polyfit(df['time_numeric'], df['Campbell_Air_Temperature'], 2) # fit polynomial
#     polynomial = np.poly1d(coefficients) #create function
#     df[f'fitted_2_Temp_Air'] = polynomial(df['time_numeric']) #values
#     df[f'Air_detrend_2_Temp'] = df['Campbell_Air_Temperature'] - df[f'fitted_2_Temp_Air']
#     df[f'Air_detrend_2_Temp'] = df[f'Air_detrend_2_Temp'] + df[f'fitted_2_Temp_Air'].median()

#     # ... and for RBRs
#     depths = ['30cm', '40cm', '50cm', '60cm', '85cm', '100cm']
#     for d in depths:
#         if df[f'RBR_Temperature_{d}'].dropna().empty:
#             print(f"{date}: Column RBR_Temperature_{d}cm is empty or contains only NaN values. creating nan-column for depth {d}.")
#         else: 
#             coefficients = np.polyfit(df['time_numeric'], df[f'RBR_Temperature_{d}'], 2) # fit polynomial
#             polynomial = np.poly1d(coefficients) #create function
#             df[f'rbr_fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
#             df[f'rbr_detrend_2_Temp_{d}'] = df[f'RBR_Temperature_{d}'] - df[f'rbr_fitted_2_Temp_{d}']
#             df[f'rbr_detrend_2_Temp_{d}'] = df[f'rbr_detrend_2_Temp_{d}'] + df[f'rbr_fitted_2_Temp_{d}'].median()

#     print(f'Mission {date} detrended.\n')
# # df_9_1 = df_9[df_9['time_numeric'] < (df_9['time_numeric'].max()/2)]
# # df_9_2 = df_9[df_9['time_numeric'] > (df_9['time_numeric'].max()/2)]

2023-03-02: Column CTD_Temperature_40cm contains only NaN values. Creating nan-column for depth 40cm.
2023-03-08: Column CTD_Temperature_50cm contains only NaN values. Creating nan-column for depth 50cm.
2023-03-09: Column CTD_Temperature_50cm contains only NaN values. Creating nan-column for depth 50cm.
2023-03-17: Column CTD_Temperature_50cm contains only NaN values. Creating nan-column for depth 50cm.


In [44]:
sum(allcruises['CTD_detrend_30cm'])

nan

In [38]:
for date in ['2023-03-02','2023-03-08','2023-03-09','2023-03-17' ]:
    df = allcruises[allcruises['Date'] == date]

    # time in total seconds
    df['time_numeric'] = (df['Date_Time'] - df['Date_Time'].min()).dt.total_seconds()  

    # same for all depths
    if date == '2023-03-08':
        depths = ['SML', '30cm', '40cm', '50cm', '60cm', '85cm']
    else:
        depths = ['SML', '30cm', '40cm', '50cm', '60cm', '85cm', '100cm']

    for d in depths:
        if df[f'CTD_Temperature_{d}'].dropna().empty:
            # If the column is empty (all NaNs), skip this depth
            print(f"{date}: Column CTD_Temperature_{d} contains only NaN values. Creating nan-column for depth {d}.")
            df[f'detrend_2_Temp_{d}'] = np.nan

        else:
            coefficients = np.polyfit(df['time_numeric'], df[f'CTD_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'detrend_2_Temp_{d}'] = df[f'CTD_Temperature_{d}'] - df[f'fitted_2_Temp_{d}']
            df[f'detrend_2_Temp_{d}'] = df[f'detrend_2_Temp_{d}'] + df[f'fitted_2_Temp_{d}'].median()
            df.rename(columns={f'CTD_Temperature_{d}' : f'CTD_Temperature_{d}_old'}, inplace=True)
            df.rename(columns={f'detrend_2_Temp_{d}' : f'CTD_Temperature_{d}'}, inplace=True)


    # ... and same for airtemp
    coefficients = np.polyfit(df['time_numeric'], df['Campbell_Air_Temperature'], 2) # fit polynomial
    polynomial = np.poly1d(coefficients) #create function
    df[f'fitted_2_Temp_Air'] = polynomial(df['time_numeric']) #values
    df[f'Air_detrend_2_Temp'] = df['Campbell_Air_Temperature'] - df[f'fitted_2_Temp_Air']
    df[f'Air_detrend_2_Temp'] = df[f'Air_detrend_2_Temp'] + df[f'fitted_2_Temp_Air'].median()

    # ... and for RBRs
    depths = ['30cm', '40cm', '50cm', '60cm', '85cm', '100cm']
    for d in depths:
        if df[f'RBR_Temperature_{d}'].dropna().empty:
            print(f"{date}: Column RBR_Temperature_{d}cm is empty or contains only NaN values. creating nan-column for depth {d}.")
        else: 
            coefficients = np.polyfit(df['time_numeric'], df[f'RBR_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'rbr_fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'rbr_detrend_2_Temp_{d}'] = df[f'RBR_Temperature_{d}'] - df[f'rbr_fitted_2_Temp_{d}']
            df[f'rbr_detrend_2_Temp_{d}'] = df[f'rbr_detrend_2_Temp_{d}'] + df[f'rbr_fitted_2_Temp_{d}'].median()

    print(f'Mission {date} detrended.\n')
# df_9_1 = df_9[df_9['time_numeric'] < (df_9['time_numeric'].max()/2)]
# df_9_2 = df_9[df_9['time_numeric'] > (df_9['time_numeric'].max()/2)]

2023-03-02: Column CTD_Temperature_40cm contains only NaN values. Creating nan-column for depth 40cm.
Mission 2023-03-02 detrended.

2023-03-08: Column CTD_Temperature_50cm contains only NaN values. Creating nan-column for depth 50cm.
2023-03-08: Column RBR_Temperature_50cmcm is empty or contains only NaN values. creating nan-column for depth 50cm.
Mission 2023-03-08 detrended.

2023-03-09: Column CTD_Temperature_50cm contains only NaN values. Creating nan-column for depth 50cm.
Mission 2023-03-09 detrended.

2023-03-17: Column CTD_Temperature_50cm contains only NaN values. Creating nan-column for depth 50cm.
Mission 2023-03-17 detrended.



In [40]:
allcruises.filter(regex = 'detren').columns.to_list()

[]

In [16]:
# average wind speed each day

for df in all_df:
    print('mean windspeed (10 m) on', df['Date'].iloc[0],': '  f'{df['Wind_Speed_U10'].mean():.2f}', 'm/s')

for df in all_df:
    print('mean windspeed (0.5m) on', df['Date'].iloc[0],': '  f'{df['Thies_Wind_Speed_CUS_AVG'].mean():.2f}', 'm/s')

mean windspeed (10 m) on 2023-03-02 : 5.12 m/s
mean windspeed (10 m) on 2023-03-03 : 6.67 m/s
mean windspeed (10 m) on 2023-03-08 : 3.13 m/s
mean windspeed (10 m) on 2023-03-09 : 2.66 m/s
mean windspeed (10 m) on 2023-03-10 : 9.03 m/s
mean windspeed (10 m) on 2023-03-11 : 8.53 m/s
mean windspeed (10 m) on 2023-03-17 : 3.96 m/s
mean windspeed (10 m) on 2023-03-18 : 3.83 m/s
mean windspeed (10 m) on 2023-03-19 : 6.12 m/s
mean windspeed (10 m) on 2023-03-20 : 6.64 m/s
mean windspeed (0.5m) on 2023-03-02 : 4.50 m/s
mean windspeed (0.5m) on 2023-03-03 : 6.13 m/s
mean windspeed (0.5m) on 2023-03-08 : 3.04 m/s
mean windspeed (0.5m) on 2023-03-09 : 2.69 m/s
mean windspeed (0.5m) on 2023-03-10 : 7.90 m/s
mean windspeed (0.5m) on 2023-03-11 : 7.47 m/s
mean windspeed (0.5m) on 2023-03-17 : 3.47 m/s
mean windspeed (0.5m) on 2023-03-18 : 3.50 m/s
mean windspeed (0.5m) on 2023-03-19 : 5.32 m/s
mean windspeed (0.5m) on 2023-03-20 : 5.83 m/s


In [ ]:
# Define FFT function
def plot_fft(signal, sampling_interval, label):
n = len(signal)
fft_values = fft(signal)
fft_freq = fftfreq(n, d=sampling_interval)
plt.plot(fft_freq[:n // 2], np.abs(fft_values)[:n // 2], label=label)